In [10]:
#####Latent Dirichlet Allocation Model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
from scipy.special import gammaln
from scipy.misc import factorial
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

####データの発生####
##データの設定
k = 15   #トピック数
d = 5000   #文書数
v  = 1000   #語彙数
w = poisson(175, d)   #1文書あたりの単語数
f = sum(w)   #総単語数

In [11]:
##パラメータの設定
#ディレクリ分布のパラメータの設定
alpha0 = np.repeat(0.2, k)   #文書のディレリ事前分布のパラメータ
alpha1 = np.repeat(0.15, v)   #単語のディレクリ事前分布のパラメータ

#ディレクリ分布からパラメータを発生
thetat = theta = np.random.dirichlet(alpha0, d)   #文書のトピック分布をディレクリ乱数から発生
phit = phi = np.random.dirichlet(alpha1, k)    #単語のトピック分布をデレクレリ乱数から発生

In [12]:
##多項分布より単語頻度行列を生成
WX = np.zeros((d, v), dtype='int')
Z = [i for i in range(d)]
vec = np.arange(0, k)

for i in range(d):
    #文書のトピックを生成
    z = multinomial(1, theta[i, :], w[i])   #文書のトピック分布を発生
    z_vec = np.dot(z, vec)

    #トピック割当から単語を生成
    word = np.zeros((w[i], v))
    for j in range(w[i]):
        word[j, :] = multinomial(1, phi[z_vec[j], :], 1)

    WX[i, :] = np.sum(word, axis=0)
    Z[i] = z_vec

In [13]:
####トピックモデルのためのデータと関数の準備####
##それぞれの文書中の単語の出現をベクトルに並べる
##データ推定用IDを作成
ID_list = [i for i in range(d)]
wd_list = [i for i in range(d)]

#文書ごとに求人IDおよび単語IDを作成
for i in range(d):
    ID_list[i] = np.repeat(i, w[i])
    num1 = (WX[i, :] > 0) * np.arange(1, v+1)
    num2 = num1[num1!=0]
    W1 = WX[i, WX[i, :] > 0]
    number = np.repeat(num2, W1)
    wd_list[i] = number

#リストをベクトル変換
wd = np.zeros(sum(w), dtype='int')
ID_d = np.zeros(sum(w), dtype='int')
start = 0

for i in range(d):
    wd[start:start+w[i]] = wd_list[i] - 1
    ID_d[start:start+w[i]] = ID_list[i]
    start += w[i] 

In [14]:
##インデックスを作成
doc_list = [i for i in range(d)]
doc_vec = [i for i in range(d)]
word_list = [i for i in range(v)]
word_vec = [i for i in range(v)]
index = np.array(range(f))

for i in range(d):
    doc_list[i] = index[ID_d==i]
    doc_vec[i] = np.repeat(1, doc_list[i].shape)
for i in range(v):
    word_list[i] = index[wd==i]
    word_vec[i] = np.repeat(1, word_list[i].shape)

In [15]:
####マルコフ連鎖モンテカルロ法でLDAのパラメータをサンプリング####
##単語ごとに尤度と負担率を計算する関数
def burden_fr(f, theta, phi, wd, w, k):
    Bur = np.zeros((f, k))
    Bur0 = np.zeros((f, k))

    #負担係数を計算
    for j in range(k):
        Bi = np.repeat(theta[:, j], w) * phi[j, wd]
        Bur[:, j] = Bi

    #負担率の分母部分
    Bur_sums = np.sum(Bur, axis=1)
    for l in range(k):
        Bur0[:, l] = Bur_sums 

    #負担率と混合率を計算
    Br = Bur / Bur0   #負担率の計算
    r = np.sum(Br, axis=0) / np.sum(Br)   #混合率の計算
    return Br, Bur, r

In [16]:
##アルゴリズムの設定
#更新ステータス
LLo = -100000000
LLw = LLo
iter = 1
dl = 100   #EMステップでの対数尤度の差の初期値
tol = 0.5
abs(dl)

100

In [17]:
##事前分布の設定
alpha1 = 0.1
beta1 = 0.1

##パラメータの初期値
mu = np.sum(WX, axis=0)/np.sum(WX)
phi = np.random.dirichlet(np.repeat(1, v), k)
theta = np.random.dirichlet(np.repeat(1, k), d)

In [18]:
##LDAのパラメータを更新
while abs(dl) >= tol:

    ##単語トピックを生成
    #トピック割当の潜在確率の期待値を推定
    out = burden_fr(f, theta, phi, wd, w, k)
    Zi = out[0]
    Zi_T = Zi.T

    ##パラメータを更新
    #トピック分布thetaの期待値を推定
    wsum = np.zeros((d, k))
    for i in range(d):
        wsum[i :] = np.dot(Zi_T[:, doc_list[i]], doc_vec[i]) + alpha1   #ディクレリ分布のパラメータ
    theta = wsum / np.repeat(np.sum(wsum, axis=1), k).reshape((d, k))   #ディクレリ分布の期待値

    #単語分布phiの期待値を推定
    vsum = np.zeros((k, v))
    for j in range(v):
        vsum[:, j] = np.dot(Zi_T[:, word_list[j]], word_vec[j]) + beta1   #ディクレリ分布のパラメータ
    phi = vsum / np.repeat(np.sum(vsum, axis=1), v).reshape((k, v))   #ディクレリ分布の期待値

    #対数尤度の更新
    LLS = np.sum(np.log(np.sum(out[1], axis=1)))   #対数尤度を計算
    iter <- iter+1
    dl = LLS-LLo
    LLo = LLS
    LLw <- np.array((LLw, LLo))
    print(LLo)

-6099211.644360539
-5890244.204075026
-5882763.8465366075
-5877152.526444978
-5872254.058010995
-5867384.345552314
-5862010.137097292
-5855619.84968386
-5847676.355362909
-5837630.564632349
-5825001.8816377865
-5809525.265396597
-5791323.384758549
-5770996.387186842
-5749502.144912962
-5727847.364729562
-5706819.046256219
-5686914.965636664
-5668386.034964067
-5651284.141640294
-5635533.0383971725
-5621024.652965832
-5607689.8504888825
-5595520.988357752
-5584577.262616271
-5574972.876684146
-5566796.814457374
-5560012.577608153
-5554472.36227098
-5549982.555318139
-5546344.185488024
-5543378.30829341
-5540934.994069135
-5538891.801121463
-5537148.94592908
-5535622.978052624
-5534240.863405594
-5532935.60718951
-5531643.085630821
-5530300.626339057
-5528848.908103016
-5527239.073375463
-5525445.85953509
-5523482.928486122
-5521409.348218458
-5519318.846629886
-5517322.376111658
-5515530.5125390235
-5514017.381695975
-5512791.889769014
-5511816.70409443
-5511034.787830061
-5510384.35567